# Contrast and Image Enhancement
## Transfer Functions
stough 202-

1. [Windowing and Piece-wise Linear Transforms](#windowing)
1. [Log Transforms](#log)
1. [Power Transforms](#power)

*A lot of pictures from my slides might go here.*

- Enhancement: manipulating an image to be more suitable *for the application*
  - Human perception
  - Compressibility
  - ?
- Contrast: difference in color or luminance between nearby elements or over a whole image
  - quantifiable globally through histograms
  - think of the difference between the Gaussian and Uniform distributions and how that looked in the RGB cube.
  
In this notebook we'll consider some images that could be *enhanced* by improving the *contrast*. We'll apply Transfer functions of various type, which can all very generally be expressed as

\begin{equation*}
s = \mathbf{T}(r)
\end{equation*}

where $r$ is the image intensity stored in the file, and $s$ is a *corrected* or transformed intensity that we want the display device to use. 

## Imports
Note the addition of the `vis_pair` function, which can nicely plot an original and changed image for side-by-side comparison.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, FloatSlider

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

import matrix_utils
from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists,
                       vis_pair,
                       lab_uniform)

<a id='windowing'></a>
## Windowing and Piece-wise Linear Transforms

The simplest transfer function one might imagine is the linear function

\begin{equation*}
\mathbf{T}(r) = Ax+B
\end{equation*}

where $A$ and $B$ are constants. Let's consider a particular case.

In [ ]:
I = plt.imread('../dip_pics/got_tooDark.jpg')
vis_hists(I)

This image has pretty much all of its pixels bunched up in the lower half of the nominal display range $[0,255]$. Notwithstanding the director's intent in choreographing the shot, let's "correct" for my old eyes. A simple linear function might do. But we have to remember that `matplotlib` expects three channel-images to stay in $[0,255]$ (if integer type).

In [ ]:
def simpleLinear(I, A = 1.0, B = 0.0):
    '''
    simpleLinear(I, A = 1.0, B = 0.0): return a linearly transformed image with the 
    provided constants A and B. 
    '''
    Tr = lambda x: A*x + B
    J = Tr(I.astype('float'))
    return np.clip(J, 0, 255).astype('uint8')

We can even see practically what this simple linear transform does in transforming the input intensity.

In [ ]:
x = np.arange(256)
plt.figure(figsize=(4,3))
plt.plot(x, x, label='equal')
plt.plot(x, simpleLinear(x, A = 3), label = '3x+0')
plt.xlabel('Input Intensity')
plt.ylabel('Output Intensity')
plt.legend()
plt.tight_layout()

In [ ]:
I_corrected = simpleLinear(I, A = 3)
vis_pair(I, I_corrected)

You can see what we did by viewing the histograms, or even (cooler), seeing what happened in the rgb cube!

In [ ]:
vis_hists(I)
vis_hists(I_corrected)

In [ ]:
vis_rgb_cube(I)
vis_rgb_cube(I_corrected)

One approach you may think of is to renormalize the intensity so that min, max is exactly $[0,255]$. This is of course *also a linear transform*:

\begin{equation*}
\mathbf{T}(r) = 255 \frac{r - \tt{I.min}}{\tt{I.max} - \tt{I.min}} \\
\end{equation*}

or $A = \frac{255}{\tt{I.max} - \tt{I.min}}$, and $B = \frac{-255\times\tt{I.min}}{\tt{I.max} - \tt{I.min}}$

In [ ]:
I_normed = simpleLinear(I, A = 255.0/(I.max()-I.min()), B = -255*I.min()/(I.max()-I.min()))
vis_pair(I_corrected, I_normed)

If we plot all of these potential linear maps we can see their varying effect.

In [ ]:
x = np.arange(256)
plt.figure(figsize=(4,3))
plt.plot(x, x, label='equal')
plt.plot(x, simpleLinear(x, A = 3), label = '3x+0')
plt.plot(x, simpleLinear(x, A = 255.0/(I.max()-I.min()), B = -255*I.min()/(I.max()-I.min())), label='min-max normed')
plt.xlabel('Input Intensity')
plt.ylabel('Output Intensity')
plt.legend()
plt.tight_layout()

### Gaussian Noise Example

We'll do one more example with windowing. We'll use a Gaussian distributed random image that we've come to know from previous notebooks. It abstractly represents poor contrast, because its intensities are bunched up relative to a uniform distributed image.

In [ ]:
I_uniform = np.uint8(256*np.random.random((100,100,3)))
I_gauss = np.clip(128 + 30*np.random.randn(100,100,3), 0, 255).astype('uint8')

In [ ]:
vis_pair(I_uniform, I_gauss, first_title='Uniform Dist', second_title='Gaussian Dist')
vis_rgb_cube(I_gauss)
vis_hists(I_gauss, bins=np.arange(257))

Looking at the histogram of `I_gauss`, you see that most of the intensities seem to be bunched up in the $[50,200]$ range. But our output range is $[0,255]$, so we have some room to work with. We want to define a transfer function that will dedicate the entire output range to where in the input range our data actually are. Let's make a function that linearly maps some input range to some output range. While we could formulate this in terms of [$Ax+B$](http://www.webmath.com/equline1.html) or the [two point form](https://www.cuemath.com/geometry/two-point-form/), I like to think of it as mixing: we're mixing between 0 and 255, and the mixing amount is equal to how far we are along the road from 50 to 200.

\begin{equation*}
\mathbf{T}(r) = (1-\alpha)\times0 + \alpha\times255\\
\alpha = \frac{r - 50}{200-50}
\end{equation*}

In [ ]:
def make_linmap(inputrange, outputrange):
    a,b = inputrange
    c,d = outputrange
    
    return lambda x: (1-((x-a)/(b-a)))*c + ((x-a)/(b-a))*d

<a id='log'></a>
## Log Transforms

Windowing redistributed all of the output range to 

<a id='power'></a>
## Power Transforms